In [ ]:
import os, sys
import pathlib
import time
import pickle

import numpy as np
import cv2

from scipy.linalg import null_space
# import sympy

import matplotlib.pyplot as plt

In [ ]:
sys.path

In [ ]:
%load_ext autoreload
%autoreload 2

if not os.path.abspath("../") in sys.path:
    sys.path.append(os.path.abspath("../"))
    sys.path.append(os.path.abspath("../src"))

from src import normalization
from src import Camera, metashape_Camera
from src import camera_correspondence, FF_mat, epipole_angle, pair_and_key_gen, coll_dict_gen
from src import pt_pair, FR_frags, FR_check
from src import min_dist, tri, reprojection_gen, gen_support_dict, gen_support
from src import all_D, frag_list_fraged, all_sep

## パイプライン

In [ ]:
cam_list = [metashape_Camera(i) for i in range(0,264,20)]
for i in range(len(cam_list)):
    cam_list[i].img_load()
    cam_list[i].contour_load()
    cam_list[i].label_load()
    cam_list[i].correspondence_contour()
    cam_list[i].para_load()
    

In [ ]:
# エピポール取得
cam_pairs = camera_correspondence(cam_list)

epipole_dict = {i:[] for i in range(len(cam_list))}
cam_pairs_F = {}
for i in cam_pairs:
    epipole1, epipole2, F = FF_mat(cam_list[i[0]].A, cam_list[i[1]].A, cam_list[i[0]].Rt, cam_list[i[1]].Rt)
    epipole_dict[i[0]].append(normalization(epipole1))
    epipole_dict[i[1]].append(normalization(epipole2))
    cam_pairs_F[i] = F

In [ ]:
# フラグメントのリストを作る
for i in range(len(cam_list)):
    im_del_list = all_D(epipole_angle(i, epipole_dict, cam_list=cam_list))# im_del_list[color][contour][del_idx]
    newCon = all_sep(cam_list[i].contour_list, im_del_list)# newCon[color][fragment][coordination]
    cam_list[i].frag_list = frag_list_fraged(newCon)

In [ ]:
#cnum = 3
#new_img = np.zeros((cam_list[0].img.shape[0],cam_list[0].img.shape[1]),dtype=np.bool8)

#for j in range(len(cam_list[cnum].frag_list)):
#    for i in range(len(cam_list[cnum].frag_list[j])):
#        curve = cam_list[cnum].frag_list[j][i][~np.isnan(cam_list[cnum].frag_list[j][i])].reshape((-1,2)).astype(int)
#        new_img[curve[:,1],curve[:,0]]=True

In [ ]:
#%matplotlib inline
#fig = plt.figure(figsize = (20, 20))
#fig.patch.set_alpha(0.)
## 3DAxesを追加
#ax = fig.add_subplot(111)
#plt.imshow(new_img)

## ペア生成

In [ ]:
%%time
import joblib
result = joblib.Parallel(n_jobs=-1)(joblib.delayed(pair_and_key_gen)(i, cam_list=cam_list, cam_pairs_F=cam_pairs_F) for i in cam_pairs_F)

In [ ]:
pair_list = {}
for i in result:
    pair_list.update(i)

In [ ]:
#pair_list[key][color][curve][pair]

In [ ]:
%%time
result = joblib.Parallel(n_jobs=-1)(joblib.delayed(coll_dict_gen)(
    i, pair_list=pair_list, cam_list=cam_list, cam_pairs_F=cam_pairs_F
) for i in pair_list)


In [ ]:
coll_dict = {}
for i in result:
    coll_dict.update(i)

In [ ]:
# 点の一対一対応
pair_pt = {}
for i in coll_dict:
    im_list = []
    for col in coll_dict[i]:
        col_list = []
        for frag in col:
            f_list = []
            for each_frag in frag:
                new_pair = pt_pair(each_frag)
                f_list.append(new_pair)
            col_list.append(f_list)
        im_list.append(col_list)
    pair_pt[i] = im_list

In [ ]:
# 座標でdictを作る
coordinate_dict = {}
for i in pair_list:
    pair_coordinate = []
    part, counterpart = FR_frags(i,cam_list=cam_list)
    for part_col, cpart_col, pair_col, PtPair_col in zip(part, counterpart, pair_list[i], pair_pt[i]):
        col_list = []
        for part_frag, pair, pt_idx in zip(part_col, pair_col, PtPair_col):
            for each_pair, each_pt_idx in zip(pair, pt_idx):
                if each_pt_idx[0].size != 0:
                    col_list.append((np.array([part_frag[each_pt_idx[0]], cpart_col[each_pair][each_pt_idx[1]]])))
        pair_coordinate.append(col_list)
    coordinate_dict[i] = pair_coordinate

In [ ]:
TDlines = {}
for i, j in enumerate(coordinate_dict):
    if j[1] == "F":
        continue
    pts = coordinate_dict[j]
    P1_ori, P2_ori, F_ori = FR_check(j)
    #pt, sep_list = connect_points(pts)
    temp_TDlines = []
    for pts_col in pts:
        col_list = []
        for pt in pts_col:
            pt = np.transpose(pt, (1, 0, 2))
            F = np.broadcast_to(F_ori, (pt.shape[0], 3, 3))
            P1 = np.broadcast_to(P1_ori, (pt.shape[0], 3, 4))
            P2 = np.broadcast_to(P2_ori, (pt.shape[0], 3, 4))
            newcoords = np.array(
                list(map(min_dist, F, pt[:, 1, :], pt[:, 0, :])))
            tri_pt = np.array(
                list(map(tri, P1, P2, newcoords[:, 1, :], newcoords[:, 0, :])))
            #pts_array = sep_array(tri_pt, sep_list)
            col_list.append(tri_pt)
        temp_TDlines.append(col_list)

    with open(r"temp/{0}_{1}_{2}.TDlines".format(j[0][0], j[0][1], j[1]), "wb") as f:
        pickle.dump(temp_TDlines, f)
    #TDlines[j] = temp_TDlines
    print((i+1)/len(coordinate_dict)*100)


In [ ]:
tags = []
for i in coordinate_dict:
    tags.append(i)

In [ ]:
%%time
import joblib
result = joblib.Parallel(n_jobs=-1,verbose=10)(joblib.delayed(reprojection_gen)(tag, cam_list=cam_list) for tag in tags)

In [ ]:
size_list = []
for tag in tags:
    size_list.append(os.path.getsize("temp/{0}_{1}_{2}.TDlines".format(tag[0][0],tag[0][1],tag[1])))

large_tags = np.array(tags)[np.where(
    np.array(size_list) >= sum(size_list)/len(size_list))]

small_tags = np.array(tags)[np.where(np.array(size_list)<sum(size_list)/len(size_list))]

In [ ]:
%%time
import joblib
result = joblib.Parallel(n_jobs=-1,verbose=10)(joblib.delayed(gen_support)(small_tags, cam_list=cam_list) for tag in tags)

In [ ]:
import joblib
% % time
result = joblib.Parallel(n_jobs=4, verbose=10)(
    joblib.delayed(gen_support)(large_tags) for tag in large_tags)


In [ ]:
sup_th = 5# サポート数
curve_fragment = []
for tag in TDlines:
    if tag[1] == "R":
        continue
    lines_list = TDlines[tag]
    with open(r"temp/{0}_{1}_{2}.support_dict".format(tag[0][0],tag[0][1],tag[1]), 'rb') as f:
        support_list, support_ac = pickle.load(f)
    #support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
    tag_list = []
    for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
        col_curve_fragment = []
        for frag, sup, sup_ac in zip(col, sup_col, ac_col):
            if sup > sup_th:
                frag = np.reshape(frag,(-1, 3))
                frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                col_curve_fragment.append(frag)
        tag_list.append(col_curve_fragment)
    curve_fragment.append(tag_list)

In [ ]:
cfs = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
for cols in curve_fragment:
    for i in range(len(cols)):
        if i == 0:
            cfs[0] += cols[i]
        if i == 1:
            cfs[1] += cols[i]
        if i == 2:
            cfs[2] += cols[i]
        if i == 3:
            cfs[3] += cols[i]
        if i == 4:
            cfs[4] += cols[i]
        if i == 5:
            cfs[5] += cols[i]
        if i == 6:
            cfs[6] += cols[i]
        if i == 7:
            cfs[7] += cols[i]
        if i == 8:
            cfs[8] += cols[i]
        if i == 9:
            cfs[9] += cols[i]
        if i == 10:
            cfs[10] += cols[i]
        if i == 11:
            cfs[11] += cols[i]
        if i == 12:
            cfs[12] += cols[i]

In [ ]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

def plot_graph():
    color_arr = np.array([[255,0,0],[0,255,0],[0,0,255],
                             [255,255,0],[255,0,255],[0,255,255],
                             [127,127,127],[127,0,127],[0,127,127]],dtype = np.int16)
    j = 0
    for i in range(len(cfs)):
        for frag in cfs[i]:
            if frag.size == 0:
                continue
            #print(frag)
            
            
            x = frag[:,0]
            y = frag[:,1]
            z = frag[:,2]
            data = [x,y,z]
            try:
                tck, u= interpolate.splprep(data, k=3)
            except ValueError:
                pass
            except TypeError:
                pass
            new = interpolate.splev(u, tck, der=0)
            c = color_arr[i]/255
            if j % 100 == 0:
                ax.plot(new[0], new[2], new[1],"-",color=c)
            
            j+=1
            
        
%matplotlib notebook
fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))
#ax.set_xticks(np.arange(-4,4,1))
#ax.set_yticks(np.arange(-4,4,1))
#ax.set_zticks(np.arange(-2,9,1))

plot_graph()

In [ ]:
import itertools

import open3d as o3d

flatten_curve_frag = np.array(list(itertools.chain.from_iterable(_curve_fragment)))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(flatten_curve_frag)

o3d.visualization.draw_geometries([pcd])

from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))

def plot_graph():
    i = 0
    for c_f in curve_fragment:
        for frag in c_f:
            if frag.size == 0:
                continue
            #print(frag)
            
            if i%100 == 0:
                x = frag[:,0]
                y = frag[:,1]
                z = frag[:,2]
                data = [x,y,z]
                try:
                    tck, u= interpolate.splprep(data, k=3)
                except ValueError:
                    pass
                except TypeError:
                    pass
                new = interpolate.splev(u, tck, der=0)

                ax.plot(new[0], new[2], new[1],"-")
                    #ax.plot(x, z, y,"-")
            i += 1

def plt_graph3d(angle):
    ax.view_init(azim=angle*5)
    
# アニメーションを作成
ani = FuncAnimation(
    fig,
    func=plt_graph3d,
    frames=72,
    init_func=plot_graph,
    interval=200
)

# imagemagickで作成したアニメーションをGIFで書き出す
ani.save("48_rolling.gif", writer="pillow", savefig_kwargs={'transparent': True, 'facecolor': 'none'})
#ani.save('48_anim.mp4', writer="ffmpeg", savefig_kwargs={'transparent': True, 'facecolor': 'none'})

In [ ]:
import pickle
with open("one_hole_leaf.curves","wb") as f:
    pickle.dump(curve_fragment, f)